In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas  as pd
from datetime import datetime, timedelta
import pytz
import time
import json

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from dateutil.relativedelta import relativedelta
# Create plotly visualization
import plotly.graph_objects as go

import epint as ep
print(ep.__version__)

In [3]:
from datetime import datetime
import pytz

# Get Istanbul time
istanbul_tz = pytz.timezone('Europe/Istanbul')

# Get the current time in Istanbul timezone
istanbul_time = datetime.now(istanbul_tz)

istanbul_time

In [4]:
# ep.set_auth("ahmetfaruk.kornaz@yepas.com", "Kpyrba2321!")

# # Verilen uevcb_list ID'lerine göre santral bilgilerini filtrele
# uevcb_ids = [
#     3197267,
#     3205381,
#     3205710,
#     134405,
#     3205527,
#     3204758,
#     3204759,
#     1740316,
#     924,
#     928,
#     3204385,
#     3204399,
#     3204400,
#     24604,
#     3194367,
#     3205524,
#     3205525,
#     2543,
#     945,
#     983,
#     3195727,
#     3206732,
#     3206733,
#     301420,
#     472111,
# ]

In [5]:
date_selecter = None

In [6]:
use_default_date = False

In [7]:
date_picker = [None, None]

In [8]:
# Önceki değerleri saklamak için değişkenler (eğer yoksa başlat)
if 'previous_date_selecter' not in globals():
    previous_date_selecter = None
if 'previous_date_picker' not in globals():
    previous_date_picker = [None, None]

now = datetime.now()

if use_default_date:
    end_date = datetime(now.year, now.month, now.day) + timedelta(days=1)
    start_date = end_date - timedelta(days=2)
else:
    end_date = datetime(now.year, now.month, now.day) + timedelta(days=1)
    start_date = end_date - timedelta(days=5)
    # date_selecter değişti mi kontrol et
    if date_selecter != previous_date_selecter:
        previous_date_selecter = date_selecter
        
        if date_selecter == 'bugun':
            start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Bugün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
        
        elif date_selecter == 'yarin':
            start_date = (datetime.now() + timedelta(1)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(2)).replace(hour=0, minute=0, second=0, microsecond=0)
            
        elif date_selecter == 'dun':
            start_date = (datetime.now()- timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now()).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Dün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")

        elif date_selecter == 'bugun ve yarin':
            start_date = (datetime.now()).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Dün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")

        elif date_selecter == 'dun ve bugun':
            start_date = (datetime.now() - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Dün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'bu hafta':
            today = datetime.now()
            start_date = (today - timedelta(days=today.weekday())).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(hours=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Bu Hafta: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'gecen hafta':
            today = datetime.now()
            start_date = (today - timedelta(days=today.weekday() + 7)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (today - timedelta(days=today.weekday() + 1)).replace(hour=23, minute=59, second=59, microsecond=999999)
            print(f"Geçen Hafta: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'bu ay':
            today = datetime.now()
            start_date = today.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Bu Ay: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'gecen ay':
            today = datetime.now()
            first_day_this_month = today.replace(day=1)
            last_day_last_month = first_day_this_month - timedelta(days=1)
            start_date = last_day_last_month.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
            end_date = (last_day_last_month + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Geçen Ay: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'son 1 hafta':
            start_date = (datetime.now() - timedelta(days=7)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Son 7 Gün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
            
        elif date_selecter == 'son 1 ay':
            start_date = (datetime.now() - timedelta(days=30)).replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Son 30 Gün: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")     
        else:
            # Varsayılan: bugün
            start_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
            end_date = (datetime.now() + timedelta(1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(f"Varsayılan (Bugün): {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
    
    # date_picker değişti mi kontrol et
    elif date_picker != previous_date_picker:
        previous_date_picker = date_picker.copy() if isinstance(date_picker, list) else date_picker
        
        if (date_picker[1] is None) and (date_picker[0] is None):
            end_date = datetime(now.year, now.month, now.day) + timedelta(days=1)
            start_date = end_date - timedelta(days=2)
            print(f"Date Picker (Varsayılan): {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
        else:
            start_date = datetime(date_picker[0].year, date_picker[0].month, date_picker[0].day, 0, 0)
            end_date = datetime(date_picker[1].year, date_picker[1].month, date_picker[1].day, 0, 0) + timedelta(days=1)
            print(f"Date Picker: {start_date.strftime('%Y-%m-%d')} - {end_date.strftime('%Y-%m-%d')}")
    else:
        # Hiçbir değişiklik yoksa, mevcut değerleri koru
        print("Tarih değişmedi, mevcut değerler korunuyor.")


start_date, end_date

In [9]:
import pandas as pd

# Read the parquet file
aic = pd.read_parquet('/work/inputs/dogalgaz/aic.parquet')
kgup = pd.read_parquet('/work/inputs/dogalgaz/kgup.parquet')
kgup_first = pd.read_parquet('/work/inputs/dogalgaz/kgup_first.parquet')


kgup = kgup[(kgup['date'] >= start_date) & (kgup['date'] < end_date)]
kgup_first = kgup_first[(kgup_first['date'] >= start_date) & (kgup_first['date'] < end_date)]
aic = aic[(aic['date'] >= start_date) & (aic['date'] < end_date)]

In [10]:
kgup

In [11]:
# today = datetime.today() + timedelta(days=1)
# version_date = today.replace(day=1)
# six_months_earlier = today #- timedelta(days=30)
# six_months_earlier = six_months_earlier.replace(day=1)

# today

In [12]:
# def split_date_by_day(date_start, date_end):
#     """
#     Verilen tarih aralığını günlük periyotlara böler.
#     Her periyot bir günü temsil eder.
#     start: mevcut gün, end: bir sonraki gün
#     """
#     periyotlar = []
    
#     # Başlangıç tarihini normalize et
#     current_start = date_start
    
#     while current_start < date_end:
#         # Her gün için bir periyot oluştur
#         # start: mevcut gün, end: bir sonraki gün
#         next_day = current_start + relativedelta(days=1)
        
#         periyotlar.append({
#             'start': current_start,
#             'end': next_day
#         })
        
#         # Bir sonraki güne geç
#         current_start = next_day
    
#     return periyotlar

# Epias Requests

onceki gun ile diff almak için once start\_date bir gun azaltılacak pivotta geri duzeltılecek

In [13]:
# start_date  = start_date - timedelta(days=1)

### service function

In [14]:
# def fetch_service(date_start, date_end, service, **params):
#     # Günlük periyotlara böl (split_date_by_day kullan)
#     periyotlar = split_date_by_day(date_start, date_end)
    
#     # Tüm günlerin verilerini birleştirmek için liste
#     all_data = []

#     for p in periyotlar:
#         # Her gün için aynı tarihi start ve end olarak kullan
#         data = service(
#             **params, 
#             startdate = p['start'].strftime('%Y-%m-%d'),
#             endtdate = p['start'].strftime('%Y-%m-%d'),
#         )
                    
#         # print(data)
#         if data and 'items' in data and data['items']:
#             df_temp = pd.DataFrame(data['items'])
#             all_data.append(df_temp)
        
#         # API rate limit için bekleme
#         time.sleep(0.4)

#     # Tüm günlerin verilerini birleştir
#     if all_data:
#         df = pd.concat(all_data, ignore_index=True)
#     else:
#         df = pd.DataFrame()
    
#     return df


In [15]:
# org_list = fetch_service(
#     start_date, 
#     end_date, 
#     ep.seffaflik_electricity.organization_list, 
#     # uevcbIds = uevcb_list
# )

# uevcb_list = fetch_service(
#     start_date, 
#     end_date, 
#     ep.seffaflik_electricity.uevcb_list_bulk, 
#     organizationIds = list(org_list['organizationId'].unique())
# )

# uevcb_list = uevcb_list.drop_duplicates()

# uevcb_list

In [16]:
# # uevcb_list DataFrame'inden bu ID'lere sahip santralları filtrele
# filtered_santraller = uevcb_list[uevcb_list['id'].isin(uevcb_ids)]

# # Dictionary oluştur: {uevcb_id: {'orgId': orgId, 'name': name}}
# santral_dict = {}
# for _, row in filtered_santraller.iterrows():
#     santral_dict[row['id']] = {
#         'orgId': row['orgId'],
#         'name': row['name']
#     }


In [17]:
# # Santral isimlerini değiştir - dict haline getir
# santral_kisa_uzun = {
#     "ERZIN": "Erzin Santrali",
#     "ALIAGA_HABAS": "ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)",
#     "BURSA_DG_1": "BURSA DG BLOK1",
#     "BURSA_DG_2": "BURSA DG BLOK2",
#     "TEKIRDAG_B": "TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)",
#     "TEKIRDAG_A": "TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)",
#     "BANDIRMA_ENJ": "ENERJİSA BANDIRMA SANTRALI",
#     "BANDIRMA_2": "BANDIRMA II DGKÇS",
#     "SAMSUN_P1": "BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)",
#     "SAMSUN_P2": "BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)",
#     "HAMITABAT_10": "HAMİTABAT ÜNİTE-10",
#     "HAMITABAT_20": "HAMİTABAT ÜNİTE-20",
#     "ANTALYA_DG": "ANTALYA ENERJİ SANTRALİ(Doğalgaz)",
#     "CENGIZ_610": "CENGİZ 610 DGKÇS",
#     "RWE_GUNEY": "RWE_TURCAS_GUNEY",
#     "YENI_KCC": "Yeni Elektrik Doğalgaz Kombine Çevrim Santrali",
#     "IC_ANADOLU": "İÇ ANADOLU DGKÇS",
#     "KIRIKKALE_ACWA": "ACWA POWER KIRIKKALE DGKÇS",
#     "ADAPAZARI": "ADAPAZARI DGKÇS",
#     "GEBZE_1": "GEBZE DGKÇS_1",
#     "GEBZE_2": "GEBZE DGKÇS_-2",
#     "ANKARA": "ANKARA DGKÇS",
#     "IZMIR_GR1": "İZMİR DGKÇS_GR1(Doğalgaz)",
#     "IZMIR_GR2": "İZMİR DGKÇS_GR2(Doğalgaz)",
# }

# santral_short_names_for_input = list(santral_kisa_uzun.keys())

In [18]:
pp_selection_input = ['']

In [19]:
# # Santral isimlerini değiştir - dict haline getir
# santral_kisa_uzun = {
#     "ERZIN": "Erzin Santrali",
#     "ALIAGA_HABAS": "ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)",
#     "BURSA_DG_1": "BURSA DG BLOK1",
#     "BURSA_DG_2": "BURSA DG BLOK2",
#     "TEKIRDAG_B": "TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)",
#     "TEKIRDAG_A": "TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)",
#     "BANDIRMA_ENJ": "ENERJİSA BANDIRMA SANTRALI",
#     "BANDIRMA_2": "BANDIRMA II DGKÇS",
#     "SAMSUN_P1": "BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)",
#     "SAMSUN_P2": "BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)",
#     "HAMITABAT_10": "HAMİTABAT ÜNİTE-10",
#     "HAMITABAT_20": "HAMİTABAT ÜNİTE-20",
#     "ANTALYA_DG": "ANTALYA ENERJİ SANTRALİ(Doğalgaz)",
#     "CENGIZ_610": "CENGİZ 610 DGKÇS",
#     "RWE_GUNEY": "RWE_TURCAS_GUNEY",
#     "YENI_KCC": "Yeni Elektrik Doğalgaz Kombine Çevrim Santrali",
#     "IC_ANADOLU": "İÇ ANADOLU DGKÇS",
#     "KIRIKKALE_ACWA": "ACWA POWER KIRIKKALE DGKÇS",
#     "ADAPAZARI": "ADAPAZARI DGKÇS",
#     "GEBZE_1": "GEBZE DGKÇS_1",
#     "GEBZE_2": "GEBZE DGKÇS_-2",
#     "ANKARA": "ANKARA DGKÇS",
#     "IZMIR_GR1": "İZMİR DGKÇS_GR1(Doğalgaz)",
#     "IZMIR_GR2": "İZMİR DGKÇS_GR2(Doğalgaz)",
# }

In [20]:
# pp_selection = []
# for pp_short_name in pp_selection_input:
#     pp_selection.append(santral_kisa_uzun[pp_short_name])

In [21]:
# for i in santral_dict:
#     print(i)

# # pp_selection içindeki herhangi bir string ile başlayan santral_dict name değerlerinin key'lerini al
# selected_uevcb_ids = []

# for uevcb_id, info in santral_dict.items():
#     santral_name = info['name']
#     # pp_selection içindeki herhangi bir string ile başlıyor mu kontrol et
#     for pp_prefix in pp_selection:
#         if santral_name.startswith(pp_prefix):
#             selected_uevcb_ids.append(uevcb_id)
#             break  # Bir eşleşme bulundu, diğer prefix'leri kontrol etmeye gerek yok

# print("\nSeçilen UEVCB ID'leri:")
# print(selected_uevcb_ids)

# uevcb_ids

In [22]:
# # selected_uevcb_ids değerlerini uevcb_ids listesinden ve santral_dict'den çıkar

# # selected_uevcb_ids'i uevcb_ids listesinden çıkar
# uevcb_ids = [id for id in uevcb_ids if id not in selected_uevcb_ids]

# # selected_uevcb_ids'i santral_dict'den çıkar
# for uevcb_id in selected_uevcb_ids:
#     if uevcb_id in santral_dict:
#         del santral_dict[uevcb_id]

# print("Güncellenmiş uevcb_ids:")
# print(uevcb_ids)
# print("\nGüncellenmiş santral_dict:")
# print(santral_dict)

### kgup son

In [23]:
# kgup = pd.DataFrame()
# for uevcb_id, pp_info in santral_dict.items():
#     # print(pp_info)
#     data = fetch_service(
#         start_date, 
#         end_date, 
#         ep.seffaflik_electricity.sbfgp, 
#         organizationId = pp_info['orgId'],
#         uevcbId = uevcb_id
#         )
    
#     data['uevcbId'] = uevcb_id
#     data['uevcbName'] = pp_info['name']
#     data['orgId'] = pp_info['orgId']
#     kgup = pd.concat([kgup, data], ignore_index=True)
# kgup

In [24]:
# kgup = fetch_service(
#     start_date, 
#     end_date, 
#     ep.seffaflik_electricity.sbfgp, 
#     uevcbIds = uevcb_ids
#     )

### kgup ilk

In [25]:
# kgup_first = pd.DataFrame()
# for uevcb_id, pp_info in santral_dict.items():
#     # print(pp_info)
#     data = fetch_service(
#         start_date, 
#         end_date, 
#         ep.seffaflik_electricity.dpp_first_version, 
#         organizationId = pp_info['orgId'],
#         uevcbId = uevcb_id
#         )
    
#     data['uevcbId'] = uevcb_id
#     data['uevcbName'] = pp_info['name']
#     data['orgId'] = pp_info['orgId']
#     kgup_first = pd.concat([kgup_first, data], ignore_index=True)

# # kgup_first = pd.DataFrame()  # Bu satır silindi - verileri kaybettiren satır
# kgup_first


# # kgup_first = fetch_service(
# #     start_date, 
# #     end_date, 
# #     ep.seffaflik_electricity.dpp-first, 
# #     uevcbIds = uevcb_ids
# #     )
# # kgup_first

### aic

In [26]:
# aic = pd.DataFrame()
# for uevcb_id, pp_info in santral_dict.items():
#     # print(pp_info)
#     data = fetch_service(
#         start_date, 
#         end_date, 
#         ep.seffaflik_electricity.aic, 
#         organizationId = pp_info['orgId'],
#         uevcbId = uevcb_id
#         )
    
#     data['uevcbId'] = uevcb_id
#     data['orgId'] = pp_info['orgId']
#     aic = pd.concat([aic, data], ignore_index=True)
# aic

### aic naming

eak df de santral isimleri yoktu uvcb\_id ıle eslestırme yapıldı

In [27]:
# # aic DataFrame'ine santral isimlerini ekle
# # uevcb_id'ye göre santral_dict'ten name bilgisini al

# aic['name'] = aic['uevcbId'].map(lambda x: santral_dict.get(x, {}).get('name', None))

# # Sonucu göster
# aic

In [28]:
# df = kgup.copy()

# aic['date'] = aic['date'].dt.tz_localize(None)
# aic = aic[['date', 'time', 'toplam', 'dogalgaz', 'orgId', 'uevcbId', 'name']]

# kgup['date'] = kgup['date'].dt.tz_localize(None)
# kgup = kgup[['date', 'time', 'toplam', 'dogalgaz', 'orgId', 'uevcbId', 'uevcbName']]


# aic

### tarıhe gore fıltrele

In [29]:
# kgup = kgup[(kgup['date'] >= start_date) & ((kgup['date'] <= end_date))]
# aic = aic[(aic['date'] >= start_date) & ((aic['date'] <= end_date))]
# aic

# Genel Bakıs

### Detaylı Santral Kguplerı

In [30]:
# # Changed index from 'time' to 'date'
# pivot_df = df.pivot_table(index='date', columns='uevcbName', values='toplam', aggfunc='sum')

# # Create the plot
# fig = go.Figure()

# # Plot each plant's data
# for column in pivot_df.columns:
#     fig.add_trace(go.Scatter(
#         x=pivot_df.index,
#         y=pivot_df[column],
#         mode='lines+markers',
#         name=column,
#         line=dict(width=2),
#         marker=dict(size=6)
#     ))

# fig.update_layout(
#     title={
#         'text': 'Santrallere Göre Günlük Toplam KGUP (2026-01)',
#         'font': {'size': 18, 'family': 'Arial, sans-serif'}
#     },
#     xaxis_title='Tarih',
#     yaxis_title='Toplam Üretim (MW)',
#     hovermode='x unified',
#     showlegend=False,
#     xaxis=dict(
#         tickangle=45,
#         showgrid=True,
#         gridcolor='lightgray',
#         gridwidth=0.5
#     ),
#     yaxis=dict(
#         showgrid=True,
#         gridcolor='lightgray',
#         gridwidth=0.5
#     ),
#     plot_bgcolor='white',
#     height=600,
#     margin=dict(l=80, r=80, t=80, b=80)
# )

# fig.show()

In [31]:
# # Group by date and uevcbName, sum toplam, and discard others
# df = df.groupby(['date'], as_index=False)['toplam'].sum()

# # First, create df_plot from the existing df
# # The df currently has 'date' column which contains datetime values
# # We need to rename it to 'datetime' for the plot
# df_plot = df.copy()
# df_plot = df_plot.rename(columns={'date': 'datetime'})

# # Create custom tick values and labels
# # We want ticks every 6 hours: 0, 6, 12, 18, 24 (which is 0 of next day)
# tick_vals = []
# tick_texts = []

# # Turkish day names
# day_names = ['Pzt', 'Sal', 'Çar', 'Per', 'Cum', 'Cmt', 'Paz']

# # Full Turkish day names for hover
# day_names_full = ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']

# for dt in pd.date_range(start=df_plot['datetime'].min(), end=df_plot['datetime'].max(), freq='12H'):
#     tick_vals.append(dt)
#     hour = dt.hour
    
#     # At midnight (hour 0), show only day name (no time)
#     if hour == 0:
#         day_name = day_names[dt.dayofweek]
#         tick_texts.append(f"{day_name}")
#     else:
#         tick_texts.append(f"{hour:02d}:00")

# # Create custom hover text
# hover_texts = []
# for idx, row in df_plot.iterrows():
#     dt = row['datetime']
#     day_name_full = day_names_full[dt.dayofweek]
#     hover_text = f"{dt.day}/{dt.month} {dt.hour:02d}:00 | {day_name_full}<br>Toplam: {row['toplam']:.1f} MW"
#     hover_texts.append(hover_text)

# # Create the plot
# fig = go.Figure()

# # Add line trace for total production over time
# fig.add_trace(go.Scatter(
#     x=df_plot['datetime'],
#     y=df_plot['toplam'],
#     mode='lines+markers',
#     name='Toplam KGUP',
#     line=dict(width=2, color='#1f77b4'),
#     marker=dict(size=4),
#     hovertext=hover_texts,
#     hoverinfo='text'
# ))

# # Add vertical lines for each Sunday (Pazar = day 6 in weekday)
# for dt in pd.date_range(start=df_plot['datetime'].min(), end=df_plot['datetime'].max(), freq='D'):
#     if dt.dayofweek == 0:  # Sunday is day 6
#         fig.add_vline(
#             x=dt,
#             line_dash="dash",
#             line_color="red",
#             line_width=1.5,
#             opacity=0.5
#         )

# fig.update_layout(
#     title={
#         'text': 'Saatlik Toplam KGUP (2026-01)',
#         'font': {'size': 20, 'family': 'Arial, sans-serif'},
#         'x': 0.5,
#         'xanchor': 'center'
#     },
#     xaxis_title='Tarih ve Saat',
#     yaxis_title='Toplam Üretim (MW)',
#     hovermode='x',
#     xaxis=dict(
#         tickangle=45,
#         showgrid=True,
#         gridcolor='lightgray',
#         gridwidth=0.5,
#         tickmode='array',
#         tickvals=tick_vals,
#         ticktext=tick_texts
#     ),
#     yaxis=dict(
#         showgrid=True,
#         gridcolor='lightgray',
#         gridwidth=0.5
#     ),
#     plot_bgcolor='white',
#     height=600,
#     margin=dict(l=80, r=80, t=100, b=100)
# )

# fig.show()

# Buradan Devam

### replace names

In [32]:
def replace_names(df, col):

    # Santral isimlerini değiştir
    santral_isimleri = [
        ("Erzin Santrali", "ERZIN"),
        ("ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)", "ALIAGA_HABAS"),
        ("BURSA DG BLOK1", "BURSA_DG_1"),
        ("BURSA DG BLOK2", "BURSA_DG_2"),
        ("TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)", "TEKIRDAG_B"),
        ("TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)", "TEKIRDAG_A"),
        ("ENERJİSA BANDIRMA SANTRALI", "BANDIRMA_ENJ"),
        ("BANDIRMA II DGKÇS", "BANDIRMA_2"),
        ("BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)", "SAMSUN_P1"),
        ("BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)", "SAMSUN_P2"),
        ("HAMİTABAT ÜNİTE-10", "HAMITABAT_10"),
        ("HAMİTABAT ÜNİTE-20", "HAMITABAT_20"),
        ("ANTALYA ENERJİ SANTRALİ(Doğalgaz)", "ANTALYA_DG"),
        ("CENGİZ 610 DGKÇS", "CENGIZ_610"),
        ("RWE_TURCAS_GUNEY", "RWE_GUNEY"),
        ("Yeni Elektrik Doğalgaz Kombine Çevrim Santrali", "YENI_KCC"),
        ("İÇ ANADOLU DGKÇS", "IC_ANADOLU"),
        ("ACWA POWER KIRIKKALE DGKÇS", "KIRIKKALE_ACWA"),
        ("ADAPAZARI DGKÇS", "ADAPAZARI"),
        ("GEBZE DGKÇS_1", "GEBZE_1"),
        ("GEBZE DGKÇS_-2", "GEBZE_2"),
        ("ANKARA DGKÇS", "ANKARA"),
        ("İZMİR DGKÇS_GR1(Doğalgaz)", "IZMIR_GR1"),
        ("İZMİR DGKÇS_GR2(Doğalgaz)", "IZMIR_GR2"),
    ]

    # Mapping dictionary oluştur
    santral_mapping = dict(santral_isimleri)

    # kgup dataframe'inde uevcbName sütununu değiştir
    df[col] = df[col].replace(santral_mapping)
    # aic['name'] = aic['name'].replace(santral_mapping)
    # kgup_first['uevcbName'] = kgup_first['uevcbName'].replace(santral_mapping)
    return df

# Pivot

In [33]:
# def make_pivot(df, value, index_col, aggfunc='sum'):
#     # Realized Generation (production_valueMWH) için pivot tablo oluşturalım
#     # Portföy ve santral tipine göre gruplandırılmış, saat sütunlarında değerler

#     # Önce dateTime'ı datetime formatına çevirelim ve saat bilgisini alalım
#     df['date'] = pd.to_datetime(df['date'])
#     # Sütun isimlerini gün/saat formatında oluştur (örn: 01/14)
#     df['hour'] = df['date'].dt.strftime('%d/%H')

#     df = replace_names(df, index_col)

#     # Realized Generation tablosu
#     table = df.pivot_table(
#         index=[index_col],
#         columns='hour',
#         values=value,
#         aggfunc=aggfunc
#     ).reset_index()

#     table.columns.name = None

#     # Toplam satırı ekleyelim
#     # Önce sayısal sütunları belirleyelim (ilk sütun uevcbName, geri kalanı saat sütunları)
#     numeric_columns = table.columns[1:]  # İlk sütun uevcbName, geri kalanı sayısal

#     # Toplam satırı oluştur - tüm sütunları bir kerede oluştur
#     total_data = {
#         index_col: 'TOPLAM',
#     }
#     # Her saat sütunu için toplamı hesapla
#     for col in numeric_columns:
#         total_data[col] = table[col].sum()
    
#     total_row = pd.DataFrame([total_data])

#     # Ortalama satırı oluştur - tüm sütunları bir kerede oluştur
#     avg_data = {
#         index_col: 'ORTALAMA',
#     }
#     # Her saat sütunu için ortalamayı hesapla
#     for col in numeric_columns:
#         avg_data[col] = table[col].mean()
    
#     avg_row = pd.DataFrame([avg_data])

#     # Tabloları birleştir
#     table = pd.concat([table, total_row, avg_row], ignore_index=True)

#     # Öncelikli sıralama listesi
#     priority_order = ['ADAPAZARI DGKÇS',
#         'İZMİR DGKÇS_GR2(Doğalgaz)',
#         'Erzin Santrali',
#         'ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)',
#         'BURSA DG BLOK1',
#         'BURSA DG BLOK2',
#         'TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)',
#         'TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)',
#         'ENERJİSA BANDIRMA SANTRALI',
#         'BANDIRMA II DGKÇS',
#         'BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)',
#         'BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)',
#         'HAMİTABAT ÜNİTE-10',
#         'HAMİTABAT ÜNİTE-20',
#         'ANTALYA ENERJİ SANTRALİ(Doğalgaz)',
#         'CENGİZ 610 DGKÇS',
#         'RWE_TURCAS_GUNEY',
#         'Yeni Elektrik Doğalgaz Kombine Çevrim Santrali',
#         'İÇ ANADOLU DGKÇS',
#         'ACWA POWER KIRIKKALE DGKÇS',
#         'GEBZE DGKÇS_1',
#         'GEBZE DGKÇS_-2',
#         'ANKARA DGKÇS',
#         'İZMİR DGKÇS_GR1(Doğalgaz)',]

#     # Sıralama için yardımcı sütun oluştur
#     def get_sort_key(plant_name):
#         # powerPlantName'i büyük harfe çevir ve karşılaştır
#         plant_name_upper = str(plant_name).upper()
#         for idx, priority_plant in enumerate(priority_order):
#             if priority_plant.upper() in plant_name_upper:
#                 return idx
#         # Listede yoksa sonlara at
#         return len(priority_order)

#     # df'den data_rows ve summary_rows'u ayır
#     # TOPLAM ve ORTALAMA satırlarını ayır
#     summary_rows = table[table[index_col].isin(['TOPLAM', 'ORTALAMA'])].copy()
#     data_rows = table[~table[index_col].isin(['TOPLAM', 'ORTALAMA'])].copy()

#     # Sıralama anahtarı ekle
#     data_rows['sort_key'] = data_rows[index_col].apply(get_sort_key)

#     # Sırala
#     data_rows = data_rows.sort_values('sort_key')

#     # sort_key sütununu kaldır
#     data_rows = data_rows.drop('sort_key', axis=1)

#     # Tabloları tekrar birleştir
#     table = pd.concat([data_rows, summary_rows], ignore_index=True)

#     # Set uevcbName as index
#     table = table.set_index(index_col)

#     table = table.T

#     # MultiIndex kolonlarından sadece ikinci seviyeyi (santral isimlerini) al
#     # table.columns = table.columns.get_level_values(1)

#     return table

In [34]:
# kgup_toplam_pivot = make_pivot(kgup, 'toplam', index_col='uevcbName')
# # dogalgaz_kgup_pivot = make_pivot(kgup, 'dogalgaz', index_col='uevcbName')
# aic_toplam_pivot = make_pivot(aic,  'toplam', index_col='name')
# kgup_first_pivot = make_pivot(kgup_first,  'toplam', index_col='uevcbName')


In [35]:
def make_pivot(df, value, index_col, aggfunc='sum'):
    # Realized Generation (production_valueMWH) için pivot tablo oluşturalım
    # Portföy ve santral tipine göre gruplandırılmış, saat sütunlarında değerler

    # Önce dateTime'ı datetime formatına çevirelim ve saat bilgisini alalım
    df['date'] = pd.to_datetime(df['date'])
    # Sütun isimlerini gün/saat formatında oluştur (örn: 01/14)
    df['hour'] = df['date'].dt.strftime('%d/%H')

    df = replace_names(df, index_col)

    # Realized Generation tablosu
    table = df.pivot_table(
        index=[index_col],
        columns='hour',
        values=value,
        aggfunc=aggfunc
    ).reset_index()

    table.columns.name = None

    # Toplam satırı ekleyelim
    # Önce sayısal sütunları belirleyelim (ilk sütun uevcbName, geri kalanı saat sütunları)
    numeric_columns = table.columns[1:]  # İlk sütun uevcbName, geri kalanı sayısal

    # Toplam satırı oluştur - tüm sütunları bir kerede oluştur
    total_data = {
        index_col: 'TOPLAM',
    }
    # Her saat sütunu için toplamı hesapla
    for col in numeric_columns:
        total_data[col] = table[col].sum()
    
    total_row = pd.DataFrame([total_data])

    # Ortalama satırı oluştur - tüm sütunları bir kerede oluştur
    avg_data = {
        index_col: 'ORTALAMA',
    }
    # Her saat sütunu için ortalamayı hesapla
    for col in numeric_columns:
        avg_data[col] = table[col].mean()
    
    avg_row = pd.DataFrame([avg_data])

    # Tabloları birleştir
    table = pd.concat([table, total_row, avg_row], ignore_index=True)

    # Öncelikli sıralama listesi
    priority_order = ['ADAPAZARI DGKÇS',
        'İZMİR DGKÇS_GR2(Doğalgaz)',
        'Erzin Santrali',
        'ALİAĞA DGKÇS(Habaş DGKÇS/DB-1)',
        'BURSA DG BLOK1',
        'BURSA DG BLOK2',
        'TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin)',
        'TEKİRDAĞ DGKÇS_A(Doğalgaz/Motorin)',
        'ENERJİSA BANDIRMA SANTRALI',
        'BANDIRMA II DGKÇS',
        'BİLGİN SAMSUN DGKÇS PT1(Doğalgaz)',
        'BİLGİN SAMSUN DGKÇS PT2(Doğalgaz)',
        'HAMİTABAT ÜNİTE-10',
        'HAMİTABAT ÜNİTE-20',
        'ANTALYA ENERJİ SANTRALİ(Doğalgaz)',
        'CENGİZ 610 DGKÇS',
        'RWE_TURCAS_GUNEY',
        'Yeni Elektrik Doğalgaz Kombine Çevrim Santrali',
        'İÇ ANADOLU DGKÇS',
        'ACWA POWER KIRIKKALE DGKÇS',
        'GEBZE DGKÇS_1',
        'GEBZE DGKÇS_-2',
        'ANKARA DGKÇS',
        'İZMİR DGKÇS_GR1(Doğalgaz)',]

    # Sıralama için yardımcı sütun oluştur
    def get_sort_key(plant_name):
        # powerPlantName'i büyük harfe çevir ve karşılaştır
        plant_name_upper = str(plant_name).upper()
        for idx, priority_plant in enumerate(priority_order):
            if priority_plant.upper() in plant_name_upper:
                return idx
        # Listede yoksa sonlara at
        return len(priority_order)

    # df'den data_rows ve summary_rows'u ayır
    # TOPLAM ve ORTALAMA satırlarını ayır
    summary_rows = table[table[index_col].isin(['TOPLAM', 'ORTALAMA'])].copy()
    data_rows = table[~table[index_col].isin(['TOPLAM', 'ORTALAMA'])].copy()

    # Sıralama anahtarı ekle
    data_rows['sort_key'] = data_rows[index_col].apply(get_sort_key)

    # Sırala
    data_rows = data_rows.sort_values('sort_key')

    # sort_key sütununu kaldır
    data_rows = data_rows.drop('sort_key', axis=1)

    # Tabloları tekrar birleştir
    table = pd.concat([data_rows, summary_rows], ignore_index=True)

    # Set uevcbName as index
    table = table.set_index(index_col)

    table = table.T

    # TOPLAM ve ORTALAMA sütunlarını en başa taşı
    cols = table.columns.tolist()
    
    # TOPLAM ve ORTALAMA'yı listeden çıkar
    if 'TOPLAM' in cols:
        cols.remove('TOPLAM')
    if 'ORTALAMA' in cols:
        cols.remove('ORTALAMA')
    
    # TOPLAM ve ORTALAMA'yı en başa ekle
    new_cols = []
    if 'TOPLAM' in table.columns:
        new_cols.append('TOPLAM')
    # if 'ORTALAMA' in table.columns:
    #     new_cols.append('ORTALAMA')
    new_cols.extend(cols)
    
    # Sütunları yeniden sırala
    table = table[new_cols]

    return table

# Pivot tablolarını yeniden oluştur
kgup_toplam_pivot = make_pivot(kgup, 'toplam', index_col='uevcbName')
# dogalgaz_kgup_pivot = make_pivot(kgup, 'dogalgaz', index_col='uevcbName')
aic_toplam_pivot = make_pivot(aic,  'toplam', index_col='name')
kgup_first_pivot = make_pivot(kgup_first,  'toplam', index_col='uevcbName')

kgup_toplam_pivot

## delta

In [36]:
def create_delta_table(realized_df, forecast_df):
    """
    Realized ve Forecast tablolarının aynı satır ve sütunlarındaki farkı (delta) hesaplar.
    Delta = Realized - Forecast
    
    Şimdiki zamandan sonraki datetime sütunları için realized değerleri 0 olarak alınır.
    
    Parameters:
    -----------
    realized_df : DataFrame
        Gerçekleşen üretim tablosu
    forecast_df : DataFrame
        Tahmin tablosu
    
    Returns:
    --------
    DataFrame
        Delta tablosu (aynı formatta)
    """
    
    # Tabloyu kopyala
    delta_table = realized_df.copy()
    
    # Şimdiki zamanı al
    now = pd.Timestamp.now() # - timedelta(hours=2)
    
    # Sayısal sütunları belirle (ilk 3 sütun index sütunları)
    numeric_columns = delta_table.columns[0:]
    
    # Her satır için delta hesapla
    for idx in range(len(delta_table)):
        for col in numeric_columns:
            # Sütun adını datetime'a çevir
            try:
                col_datetime = pd.to_datetime(col)
            except:
                col_datetime = None
            
            # Eğer sütun datetime'ı şimdiki zamandan sonraysa, delta değerini 0 olarak al
            if col_datetime is not None and col_datetime > now:
                delta_table.iloc[idx, delta_table.columns.get_loc(col)] = 0
            else:
                realized_value = realized_df.iloc[idx][col]
                forecast_value = forecast_df.iloc[idx][col]
                
                # NaN kontrolü ve delta hesaplama
                if pd.notna(realized_value) and pd.notna(forecast_value):
                    delta_table.iloc[idx, delta_table.columns.get_loc(col)] = realized_value - forecast_value
                elif pd.notna(forecast_value):
                    # Realized NaN ama forecast var ise, realized'ı 0 kabul et
                    delta_table.iloc[idx, delta_table.columns.get_loc(col)] = 0  # 0 - forecast_value
                else:
                    # Her ikisi de NaN ise 0 olarak ayarla (heatmap için)
                    delta_table.iloc[idx, delta_table.columns.get_loc(col)] = 0
    
    return delta_table

## day shift 

In [37]:
def shift_pivot(df, day, forward=True):
    """
    DataFrame'in sütun isimlerini (gün değerlerini) kaydırır, değerler aynı kalır.
    Son günü dahil etmez.
    
    Parameters:
    -----------
    df : DataFrame
        MultiIndex sütunlu DataFrame (Gün, Saat)
    day : int
        Kaydırma miktarı (gün sayısı)
    forward : bool
        True ise ileri, False ise geri kaydırma
    
    Returns:
    --------
    DataFrame
        Sütun isimleri kaydırılmış DataFrame (değerler aynı)
    """
    if forward:
        multiplier = 1
    else:
        multiplier = -1

    # DataFrame'i kopyala
    shifted_df = df.copy()
    
    # Sütun isimlerini al (MultiIndex: Gün, Saat)
    columns = df.columns
    
    # Günleri bul
    gunler = [col[0] for col in columns]
    max_gun = max(gunler)
    
    # Son günü hariç tut
    columns_to_keep = [col for col in columns if col[0] != max_gun]
    shifted_df = shifted_df[columns_to_keep]
    
    # Yeni sütun isimleri için liste oluştur
    new_columns = []
    
    for col in columns_to_keep:
        gun, saat = col
        # Günü integer'a çevir, kaydırma miktarını ekle ve tekrar string'e çevir
        yeni_gun = str(int(gun) + (multiplier * day))
        new_columns.append((yeni_gun, saat))
    
    # Yeni MultiIndex oluştur (değerler aynı kalır, sadece sütun isimleri değişir)
    shifted_df.columns = pd.MultiIndex.from_tuples(new_columns, names=['Gün', 'Saat'])
    
    return shifted_df

In [38]:
# shifted_kgup_toplam_pivot = shifted_kgup_toplam_pivot.copy()

geri düzelt kaydırma ıcın bır gun fazla almıstık o gunu geri filtrele

In [39]:
# shifted_kgup_toplam_pivot.diff(periods=24,axis=1)

# HTML

## Functions

### get\_color

In [40]:
# Function to get color based on value with automatic palette selection
def get_color(value, vmin, vmax, has_negative):
    if pd.isna(value):
        return '#f0f0f0', '#000000'  # Light gray for NaN, black text
    
    # Handle case where all values are the same
    if vmin == vmax:
        return '#ffffff', '#000000'  # White for uniform values, black text
    
    # If row has negative values, use diverging colormap (blue-white-red centered at zero)
    if has_negative:
        # Determine the max absolute value for symmetric scaling
        abs_max = max(abs(vmin), abs(vmax))
        
        if abs_max == 0:
            return '#ffffff', '#000000'  # White for all zeros, black text
        
        # Normalize value between -1 and 1 (symmetric around zero)
        normalized = value / abs_max
        
        if normalized < 0:
            # Negative values: white to blue (3 equal steps)
            intensity = abs(normalized)
            if intensity < 0.33:
                # Light blue
                r = int(255 - intensity / 0.33 * 100)
                g = int(255 - intensity / 0.33 * 50)
                b = 255
                text_color = '#000000'
            elif intensity < 0.66:
                # Medium blue
                r = int(155 - (intensity - 0.33) / 0.33 * 55)
                g = int(205 - (intensity - 0.33) / 0.33 * 105)
                b = 255
                text_color = '#000000'
            else:
                # Dark blue
                r = int(100 - (intensity - 0.66) / 0.34 * 100)
                g = int(100 - (intensity - 0.66) / 0.34 * 100)
                b = int(255 - (intensity - 0.66) / 0.34 * 55)
                text_color = '#ffffff'
        else:
            # Positive values: white to red (3 equal steps - symmetrical to negative)
            if normalized < 0.33:
                # Light red (white to light red)
                r = 255
                g = int(255 - (normalized / 0.33) * 50)
                b = int(255 - (normalized / 0.33) * 100)
                text_color = '#000000'
            elif normalized < 0.66:
                # Medium red
                r = 255
                g = int(205 - ((normalized - 0.33) / 0.33) * 105)
                b = int(155 - ((normalized - 0.33) / 0.33) * 55)
                text_color = '#000000'
            else:
                # Dark red
                r = int(255 - ((normalized - 0.66) / 0.34) * 55)
                g = int(100 - ((normalized - 0.66) / 0.34) * 100)
                b = int(100 - ((normalized - 0.66) / 0.34) * 100)
                text_color = '#ffffff'
    else:
        # Only zero or positive values: Use same blue-white-red scheme
        # Normalize value between 0 and 1
        normalized = (value - vmin) / (vmax - vmin) if vmax != vmin else 0
        
        if value == 0:
            return '#f0f0f0', '#000000'
        elif normalized < 0.5:
            print("test")
            # Below midpoint: white to blue (same as negative side)
            intensity = normalized * 2  # Scale 0-0.5 to 0-1
            if intensity < 0.33:
                # Light blue
                r = int(255 - intensity / 0.33 * 100)
                g = int(255 - intensity / 0.33 * 50)
                b = 255
                text_color = '#000000'
            elif intensity < 0.66:
                # Medium blue
                r = int(155 - (intensity - 0.33) / 0.33 * 55)
                g = int(205 - (intensity - 0.33) / 0.33 * 105)
                b = 255
                text_color = '#000000'
            else:
                # Dark blue
                r = int(100 - (intensity - 0.66) / 0.34 * 100)
                g = int(100 - (intensity - 0.66) / 0.34 * 100)
                b = int(255 - (intensity - 0.66) / 0.34 * 55)
                text_color = '#ffffff'
        else:
            # Above midpoint: white to red (same as positive side)
            intensity = (normalized - 0.5) * 2  # Scale 0.5-1 to 0-1
            if intensity < 0.33:
                # Light red
                r = 255
                g = int(255 - (intensity / 0.33) * 50)
                b = int(255 - (intensity / 0.33) * 100)
                text_color = '#000000'
            elif intensity < 0.66:
                # Medium red
                r = 255
                g = int(205 - ((intensity - 0.33) / 0.33) * 105)
                b = int(155 - ((intensity - 0.33) / 0.33) * 55)
                text_color = '#000000'
            else:
                # Dark red
                r = int(255 - ((intensity - 0.66) / 0.34) * 55)
                g = int(100 - ((intensity - 0.66) / 0.34) * 100)
                b = int(100 - ((intensity - 0.66) / 0.34) * 100)
                text_color = '#ffffff'
    
    return f'rgb({r},{g},{b})', text_color

In [41]:
# 🔥 ÖZEL KURAL: Eğer int(değer) == 0 ise renklendirme yapma
def apply_conditional_gradient(row):
    # Satırdaki tüm değerleri kontrol et
    non_zero_values = []
    for val in row:
        if pd.notna(val):
            try:
                if int(val) != 0:
                    non_zero_values.append(val)
            except (ValueError, TypeError):
                pass
    
    # Eğer tüm değerler 0 ise veya hiç non-zero değer yoksa, renklendirme yapma
    if len(non_zero_values) == 0:
        return ['background-color: #ffffff'] * len(row)
    
    # Non-zero değerler varsa, gradient uygula
    vmin = min(non_zero_values)
    vmax = max(non_zero_values)
    
    # Eğer tüm non-zero değerler aynıysa
    if vmin == vmax:
        return ['background-color: #ffffff'] * len(row)
    
    # Negatif değer var mı kontrol et
    has_negative = vmin < 0
    
    # Her hücre için renk hesapla
    colors = []
    for val in row:
        if pd.isna(val):
            colors.append('background-color: #f0f0f0')
        else:
            try:
                if int(val) == 0:
                    colors.append('background-color: #ffffff')
                else:
                    # Eğer negatif değerler varsa, diverging colormap kullan (0 merkezli)
                    if has_negative:
                        # 0'ı merkez olarak al, simetrik ölçeklendirme
                        abs_max = max(abs(vmin), abs(vmax))
                        
                        if val < 0:
                            # Negatif değerler: beyaz -> mavi
                            intensity = abs(val) / abs_max
                            # Beyazdan maviye geçiş
                            r = int(255 * (1 - intensity))
                            g = int(255 * (1 - intensity))
                            b = int(255 * (1 - intensity) + 255 * intensity)
                        else:
                            # Pozitif değerler: beyaz -> kırmızı
                            intensity = val / abs_max
                            # Beyazdan kırmızıya geçiş
                            r = int(255 * (1 - intensity) + 255 * intensity)
                            g = int(255 * (1 - intensity))
                            b = int(255 * (1 - intensity))
                    else:
                        # Sadece pozitif değerler varsa, normal gradient (min -> max)
                        normalized = (val - vmin) / (vmax - vmin)
                        # RdBu_r colormap'i simüle et (mavi -> beyaz -> kırmızı)
                        if normalized < 0.5:
                            # Mavi tonları
                            intensity = normalized * 2
                            r = int(255 * (1 - intensity) + 33 * intensity)
                            g = int(255 * (1 - intensity) + 102 * intensity)
                            b = int(255 * (1 - intensity) + 172 * intensity)
                        else:
                            # Kırmızı tonları
                            intensity = (normalized - 0.5) * 2
                            r = int(33 * (1 - intensity) + 178 * intensity)
                            g = int(102 * (1 - intensity) + 24 * intensity)
                            b = int(172 * (1 - intensity) + 43 * intensity)
                    
                    # RGB'yi HEX'e çevir
                    hex_color = '#{:02x}{:02x}{:02x}'.format(r, g, b)
                    colors.append(f'background-color: {hex_color}')
            except (ValueError, TypeError):
                colors.append('background-color: #ffffff')
    
    return colors

### excel

In [42]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import base64
from io import BytesIO

# Function to create download button for Excel with styling
def create_excel_download_button_with_style(html_content, styled_df, filename="file.xlsx"):
    # """
    # Creates a download button for Excel file with styling preserved
    # """
    # # Create Excel file in memory
    # output = BytesIO()
    # with pd.ExcelWriter(output, engine='openpyxl') as writer:
    #     # Export styled dataframe to Excel
    #     styled_df.to_excel(writer, sheet_name=f'{filename.split(".")[0]}', engine='openpyxl')
    #     # writer.close()
    
    # excel_data = output.getvalue()
    # # output.close()  # Explicitly close the BytesIO object
    # b64 = base64.b64encode(excel_data).decode()
    
    # # Create HTML button with download link
    # button_html = f'''
    # <div style="margin: 10px 0;">
    #     <a download="{filename}" 
    #        href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" 
    #        style="display: inline-block; padding: 10px 20px; background-color: #4CAF50; color: white; 
    #               text-decoration: none; border-radius: 5px; font-weight: bold; cursor: pointer;">
    #         {button_text}
    #     </a>
    # </div>
    # '''

    # 🔥 Excel butonu için SVG ikonu (Excel logo - download arrow)
    excel_icon_svg = '''<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="white" style="width: 18px; height: 18px;">
        <path d="M19 9h-4V3H9v6H5l7 7 7-7zM5 18v2h14v-2H5z"/>
    </svg>'''

    # 🔥 Excel download butonu için base64 encode
    excel_button_html = ""
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        styled_df.to_excel(writer, sheet_name=f'{filename}', engine='openpyxl')

    excel_data = output.getvalue()
    b64 = base64.b64encode(excel_data).decode()
    excel_button_html = f'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}'

    excel_button_cell = ""
    excel_button_cell = f'''
    <a href="{excel_button_html}" 
        download="{filename}" 
        style="display: flex; align-items: center; justify-content: center; width: 100%; height: 100%; 
                color: white; text-decoration: none; cursor: pointer; background-color: transparent;"
        title="Excel'e İndir"
        onmouseover="this.style.backgroundColor='#165C35';" 
        onmouseout="this.style.backgroundColor='transparent';">
        {excel_icon_svg}
    </a>
    '''

    html_content = html_content.replace('<th class="index_name level0" >uevcbName</th>', 
                                            f'<th class="index_name level0">{excel_button_cell}</th>')

    html_content = html_content.replace('<th class="index_name level0" >Gün</th>',                                          f'<th class="index_name level0">{excel_button_cell}</th>')

    return html_content

### auto width col

In [43]:
def auto_width_columns(styler):
    """
    Automatically adjust column widths based on the longest value in each column
    """
    df = styler.data
    
    # Calculate max width for each column
    col_widths = {}
    for col in df.columns:
        # Get max length of column name
        max_len = len(str(col))
        
        # Get max length of values in column
        for val in df[col]:
            val_len = len(str(val))
            if val_len > max_len:
                max_len = val_len
        
        # Add some padding and convert to pixels (approximate: 1 char ≈ 10px)
        col_widths[col] = f'10px'
    
    # Create CSS styles for column widths
    styles = []
    for i, (col, width) in enumerate(col_widths.items()):
        styles.append({
            'selector': f'th.col{i}, td.col{i}',
            'props': [('width', width), ('min-width', width)]
        })
    
    return styler.set_table_styles(styles)

### Styled Table

In [44]:
# def styled_table(df, excel_file_name, lock_excel_button=False):

    
#     # if '/' in str(df.index[0]):  # Eğer index "28 / 15" formatındaysa
#     #     # Index'i split et
#     #     index_parts = df.index.str.split('/', expand=True)
#     #     index_parts.columns = ['Gün', 'Saat']
        
#     #     # MultiIndex oluştur
#     #     df.index = pd.MultiIndex.from_arrays(
#     #         [index_parts['Gün'].astype(int), index_parts['Saat'].astype(int)],
#     #         names=['Gün', 'Saat']
#     #     )


#     # df'un index'ini multi-index formatına çevir
#     # "28/00" formatını ("28", "00") şeklinde Gün ve Saat olarak ayır

#     # Önce index'in zaten MultiIndex olup olmadığını kontrol et
#     if isinstance(df.index, pd.MultiIndex):
#         pass
#     else:
#         gun_list = []
#         saat_list = []
        
#         for idx in df.index:
#             parts = str(idx).split('/')
#             gun_list.append(int(parts[0]))
#             saat_list.append(int(parts[1]))
        
#         # MultiIndex oluştur
#         df.index = pd.MultiIndex.from_arrays(
#             [gun_list, saat_list],
#             names=['Gün', 'Saat']
#         )

#     df_columns = df.columns.to_list()

#     styled_df = df.style
#     styled_df = auto_width_columns(styled_df)
#     styled_df = styled_df.set_properties(**{'text-align': 'center'})
#     styled_df = styled_df.format(thousands=',', precision=0)
#     styled_df = styled_df.set_sticky(axis="index")

#     # def color_gradient(val, vmin, vmax):
#     #     """
#     #     Değeri normalize et ve maviden kırmızıya renk geçişi yap
#     #     """
#     #     if not isinstance(val, (int, float)):
#     #         return None
        
#     #     # Normalize et (0-1 arası)
#     #     if vmax == vmin:
#     #         norm = 0.5
#     #     else:
#     #         norm = (val - vmin) / (vmax - vmin)
        
#     #     # Mavi (0) -> Kırmızı (1) geçişi
#     #     # RGB: Mavi (0, 0, 255) -> Kırmızı (255, 0, 0)
#     #     r = int(norm * 255)
#     #     b = int((1 - norm) * 255)
#     #     g = 0
        
#     #     return f'color: rgb({r}, {g}, {b}); font-weight: bold;'

#     # # styled_df'e ekle
#     # styled_df = styled_df.map(color_values)

#     # 🔥 Her sütun için ayrı min-max ile renklendirme
#     numeric_cols = df.select_dtypes(include=[np.number]).columns
    
#     for col in numeric_cols:
#         vmin = df[col].min()
#         vmax = df[col].max()
#         styled_df = styled_df.map(
#             lambda v: color_gradient(v, vmin, vmax),
#             subset=[col]
#         )

#     styled_df = styled_df.set_table_styles([
#         # 🔥 Tablo genişliğini AUTO yerine MIN-CONTENT yap
#         {
#             'selector': 'table',
#             'props': [
#                 ('width', '100%'),
#                 ('max-width', '100%'),  # 500% yerine 100%
#                 ('border-collapse', 'collapse'),
#                 ('table-layout', 'auto')  # fixed yerine auto - resize için gerekli
#             ]
#         },
#         # 🔥 Header genişliği - daha dar
#         {
#             'selector': 'th',
#             'props': [
#                 ('max-width', 'none'),  # Limit kaldır
#                 ('min-width', '30px'),  # 50px'den 30px'e düşürdük - daha dar olabilsin
#                 # ('white-space', 'normal'),
#                 ('word-wrap', 'break-word'),
#                 ('overflow-wrap', 'break-word'),
#                 ('font-size', '12px'),
#                 ('padding', '4px'),
#                 ('line-height', '1.2'),
#                 ('background', '#34495e'),
#                 ('font-weight', '600'),
#                 ('color', 'white'),
#                 ('border-right', '1px solid #2c3e50'),
#                 ('position', 'relative'),
#                 ('user-select', 'none'),
#                 ('white-space', 'nowrap'),
#                 ('overflow', 'hidden'),
#                 ('text-overflow', 'ellipsis')
#             ]
#         },
#         # Index sütunu header'ları
#         {
#             'selector': 'th.blank',
#             'props': [
#                 ('max-width', 'none'),
#                 ('width', 'auto'),  # 10px yerine auto
#                 ('font-size', '10px')
#             ]
#         },
#         # Index değerleri
#         {
#             'selector': 'th.row_heading',
#             'props': [
#                 ('max-width', 'none'),
#                 ('min-width', '30px'),  # 60px'den 30px'e düşürdük
#                 ('width', 'auto'),
#                 ('font-size', '10px'),
#                 ('white-space', 'nowrap'),
#                 ('padding', '4px')
#             ]
#         },
#         # MultiIndex seviye 0
#         {
#             'selector': 'th.level0',
#             'props': [
#                 ('max-width', 'none'),
#                 ('min-width', '30px'),  # 50px'den 30px'e düşürdük
#                 ('font-size', '10px')
#             ]
#         },
#         # MultiIndex seviye 1
#         # {
#         #     'selector': 'th.level1',
#         #     'props': [
#         #         ('display', 'none'),
#         #         ('max-width', 'none'),
#         #         ('min-width', '50px'),
#         #         ('font-size', '10px')
#         #     ]
#         # },
#         # 🔥 Hücre genişliği - içeriğe göre
#         # Mevcut kodu bununla değiştir:
#         {
#             'selector': 'td',
#             'props': [
#                 ('max-width', 'none'),
#                 ('min-width', '20px'),
#                 ('width', 'auto'),
#                 ('white-space', 'nowrap'),
#                 ('padding', '2px 4px'),  # 0px yerine minimal padding
#                 ('line-height', '1'),     # 0.5 yerine 1
#                 ('overflow', 'hidden'),
#                 ('text-overflow', 'ellipsis'),
#                 ('word-wrap', 'break-word'),
#                 ('overflow-wrap', 'break-word'),
#                 ('font-size', '14px')
#             ]
#         },
#         # resizing
#         {
#             'selector': 'th .resizer',
#             'props': [
#                 ('position', 'absolute'),
#                 ('top', '0'),
#                 ('right', '0'),
#                 ('width', '6px'),
#                 ('height', '100%'),
#                 ('cursor', 'col-resize'),
#                 ('background', 'transparent'),
#                 ('z-index', '20')
#             ]
#         },
#         {
#             'selector': '.resizer:hover',
#             'props':[
#                 ('background', 'rgba(52, 152, 219, 0.5)')
#             ]
#         },
#         {
#             'selector': 'th.resizing',
#             'props':[
#                 ('background', '#2c3e50')
#             ]
#         }
#     ])

#     # JavaScript code as a separate string to avoid f-string issues
#     js_code = """
#         // Wait for DOM to be fully loaded
#         setTimeout(() => {
#             const table = document.querySelector('.scrollable-table table');
#             if (!table) {
#                 console.error('Table not found');
#                 return;
#             }
            
#             const headers = table.querySelectorAll('thead th');
            
#             let currentResizer = null;
#             let currentTh = null;
#             let startX = 0;
#             let startWidth = 0;

#             headers.forEach((th, index) => {
#                 // Create resizer element
#                 const resizer = document.createElement('div');
#                 resizer.className = 'resizer';
#                 resizer.style.position = 'absolute';
#                 resizer.style.top = '0';
#                 resizer.style.right = '0';
#                 resizer.style.width = '6px';
#                 resizer.style.height = '100%';
#                 resizer.style.cursor = 'col-resize';
#                 resizer.style.background = 'transparent';
#                 resizer.style.zIndex = '20';
                
#                 th.style.position = 'relative';
#                 th.appendChild(resizer);

#                 resizer.addEventListener('mousedown', (e) => {
#                     currentResizer = resizer;
#                     currentTh = th;
#                     startX = e.pageX;
#                     startWidth = th.offsetWidth;
                    
#                     currentTh.classList.add('resizing');
#                     document.body.style.cursor = 'col-resize';
#                     document.body.style.userSelect = 'none';
                    
#                     e.preventDefault();
#                     e.stopPropagation();
#                 });
#             });

#             document.addEventListener('mousemove', (e) => {
#                 if (!currentResizer) return;

#                 const diff = e.pageX - startX;
#                 const newWidth = Math.max(30, startWidth + diff);  // 50'den 30'a düşürdük - minimum genişlik
                
#                 currentTh.style.width = newWidth + 'px';
#                 currentTh.style.minWidth = newWidth + 'px';
#                 currentTh.style.maxWidth = newWidth + 'px';
#             });

#             document.addEventListener('mouseup', () => {
#                 if (currentTh) {
#                     currentTh.classList.remove('resizing');
#                 }
#                 currentResizer = null;
#                 currentTh = null;
#                 document.body.style.cursor = '';
#                 document.body.style.userSelect = '';
#             });
            
#             console.log('Column resize initialized for', headers.length, 'columns');
#         }, 100);
#     """

#     row_count = len(df)
#     row_height = 21  # ortalama satır yüksekliği (px)
#     header_height = 40  # header yüksekliği
#     visible_rows = 48

#     calculated_height = (visible_rows * row_height) + header_height

#     html_content = f'''
#     <style>
#         * {{
#             margin: 0;
#             padding: 0;
#             box-sizing: border-box;
#         }}

#         html, body {{
#             width: 100%;
#             margin: 0;
#             padding: 0;
#             overflow-x: auto;
#         }}
        
#         .table-wrapper {{
#             display: flex;
#             justify-content: center;
#             width: 100vw;
#             max-width: 100vw;
#             padding: 20px;
#             box-sizing: border-box;
#             margin-left: calc(-50vw + 50%);
#             overflow-x: auto;
#             overflow-y: auto;
#             max-height: 1500px;  /* 🔥 800px → 1500px */
#         }}
        
#         .scrollable-table {{
#             max-height: {calculated_height}px;  /* 🔥 600px → 1400px (48 satır için) */
#             overflow-y: auto;
#             overflow-x: auto;
#             border: 1px solid #ddd;
#             border-radius: 5px;
#             width: fit-content;
#             max-width: calc(100vw - 40px);
#             margin: 0 auto;
#         }}
        
#         .scrollable-table table {{
#             border-collapse: collapse;
#             width: auto;
#             table-layout: auto;
#         }}
        
#         /* İkinci header'ı tamamen gizle */
#         .scrollable-table thead tr:nth-child(2) {{
#             display: none !important;
#             height: 0 !important;
#             line-height: 0 !important;
#         }}
#         .scrollable-table thead tr:nth-child(2) th {{
#             display: none !important;
#             height: 0 !important;
#             padding: 0 !important;
#             margin: 0 !important;
#             border: none !important;
#         }}
        
#         .scrollable-table thead tr:first-child {{
#             position: sticky;
#             top: 0;
#             z-index: 12;
#         }}
        
#         .scrollable-table thead th {{
#             background-color: #2c3e50;
#             color: white;
#             font-weight: bold;
#             padding: 6px;
#             box-shadow: 0 2px 2px -1px rgba(0, 0, 0, 0.4);
#             border: 1px solid #34495e;
#             line-height: 1.2;
#             white-space: nowrap;
#             position: relative;
#         }}
        
#         .scrollable-table tbody td {{
#             background-color: white !important;
#             border-left: 2px solid #444444;
#             border-right: 2px solid #444444;
#             border-top: 1px solid #ffffff;
#             border-bottom: 1px solid #ffffff;
#             padding: 2px 4px !important;  # 0px yerine
#             line-height: 1.2 !important;  # 0.5 yerine
#             height: auto !important;      # 25px yerine auto
#             white-space: nowrap;
#             font-weight: bold;
#         }}
        
#         .scrollable-table tbody tr:hover {{
#             background-color: #f5f5f5;
#         }}
        
#         /* 🔥 STICKY INDEX CSS - Sadece position ve z-index, renk/boyut YOK */
#         .scrollable-table thead th.row_heading,
#         .scrollable-table thead th.blank,
#         .scrollable-table thead th.index_name,
#         .scrollable-table tbody th {{
#             position: sticky !important;
#             left: 0 !important;
#         }}
        
#         /* Resizer styles */
#         .scrollable-table th .resizer {{
#             position: absolute;
#             top: 0;
#             right: 0;
#             width: 6px;
#             height: 100%;
#             cursor: col-resize;
#             background: transparent;
#             z-index: 20;
#         }}
        
#         .scrollable-table th .resizer:hover {{
#             background: rgba(52, 152, 219, 0.5);
#         }}
        
#         .scrollable-table th.resizing {{
#             background: #2c3e50;
#         }}
#     </style>
#     <script>
#         {js_code}
#     </script>
#     <div class="table-wrapper">
#         <div class="scrollable-table">
#             {styled_df.to_html()}
#         </div>
#     </div>
#     '''

#     if not lock_excel_button:
#         html_content = create_excel_download_button_with_style(html_content, styled_df, "test.xlsx")
#     display(HTML(html_content))
#     return styled_df

In [45]:
# def color_gradient(val, vmin, vmax):
#     """
#     Maksimuma yüzde olarak yakınlığa göre renklendirme
#     - Maksimumun %80-85'inden yukarısı -> koyu mavi
#     - Altı -> kırmızı
#     """
#     if not isinstance(val, (int, float)):
#         return None
    
#     # 0 ise beyaz
#     if round(val) == 0:
#         return 'color: #ffffff; font-weight: bold;'
    
#     # Değer aralığını hesapla
#     value_range = vmax - vmin
    
#     # Küçük aralıklar için hepsini koyu mavi yap
#     if value_range < 20:
#         return 'color: #0a305d; font-weight: bold;'
    
#     # Maksimumun yüzde kaçında
#     percentage_of_max = (val / vmax) * 100


#     value_range = vmax - vmin
#     # threshold_pct = dynamic_percentage(value_range, base=10, multiplier=10)
#     threshold_pct=0.8

#     # Maksimumun threshold_pct'sinin üstü -> koyu mavi
#     if (val / vmax) * 100 >= (100 - threshold_pct):
#         return 'color: #0a305d; font-weight: bold;'
#     else:
#         return 'color: #e61010; font-weight: bold;'

#     # # %85'in üstü -> koyu mavi (850/1000, 425/500, 85/100)
#     # if percentage_of_max >= 85:
#     #     return 'color: #0a305d; font-weight: bold;'  # Koyu mavi
#     # elif percentage_of_max >= 70:
#     #     return 'color: #0a305d; font-weight: bold;'  # Orta mavi
#     # elif percentage_of_max >= 20:
#     #     return 'color: #0a305d;; font-weight: bold;'  # Açık mavi
#     # else:
#     #     return 'color: #e61010; font-weight: bold;'  # Kırmızı 

In [84]:
def color_gradient_to_prev_max_vmax(val, prev_val, next_val, vmax):
    """
    Yüzdelik değişime göre renklendirme
    """
    if not isinstance(val, (int, float)) or not isinstance(prev_val, (int, float)):
        return ''
    
    # Önce val == 0 kontrolü
    if val == 0 or round(val, 2) == 0:
        return 'color: rgb(255, 255, 255); font-weight: bold;'  # white
        
    
    # prev_val == 0 ise renklendirme yapma - FIX: == yerine = kullanılmalı
    if prev_val == 0:
        prev_val = 1  # Assignment operator (=) instead of comparison (==)
    if next_val == 0:
        next_val = 1  # Assignment operator (=) instead of comparison (==)
    
    prev_change_pct = abs(((val - prev_val) / abs(prev_val)) * 100)
    next_change_pct = abs(((val - next_val) / abs(next_val)) * 100)
    vmax_change_pct = abs(((val - vmax) / abs(vmax)) * 100)

    prev_change_amnt = abs(val-prev_val)
    next_change_amnt = abs(val-next_val)
    vmax_change_amnt = abs(val-vmax)


    if vmax_change_pct > 70:
        return 'color: rgb(255, 0, 0); font-weight: bold;'

    if vmax_change_pct>20:
        if (next_change_pct>25) and (prev_change_pct>25):
            return 'color: rgb(255, 0, 0); font-weight: bold;'
        elif prev_change_pct>45:
            return 'color: rgb(255, 0, 0); font-weight: bold;'
        elif next_change_amnt > 10 or prev_change_amnt > 150:
            return 'color: rgb(255, 0, 0); font-weight: bold;'
        else:
            return 'color: rgb(10, 48, 93); font-weight: bold;'
    else:
        return 'color: rgb(10, 48, 93); font-weight: bold;'

color_gradient_to_prev_max_vmax(8750, 7500, 10100, 12500)

In [483]:
import numpy as np

def color_gradient(val, data_list):


    data_list = [x for x in data_list if x < 0 or x >= 1]

    # val == 0 kontrolü
    if val == 0 or round(val, 2) == 0:
        return 'color: rgb(255, 255, 255); font-weight: bold;'  # beyaz

    base_min = 100
    vmax = max(data_list)
    vmin = min(data_list)
    vavg = sum(data_list) / len(data_list)

    if vmin < base_min:
        vmin=base_min


    if vmin*1.3 > vavg*0.8:
        vmin_upper = vavg*0.8
    else:
        vmin_upper = vmin*1.3
    
    # vmin_upper = vavg*0.9 if  else vmin*1.3

    if val <= vmin_upper:
        return 'color: rgb(255, 0, 0); font-weight: bold;' 
    else:
        return 'color: rgb(10, 48, 93); font-weight: bold;' 

In [504]:
import numpy as np

def styled_table(df, excel_file_name, lock_excel_button=False):
    if isinstance(df.index, pd.MultiIndex):
        pass
    else:
        gun_list = []
        saat_list = []
        
        for idx in df.index:
            parts = str(idx).split('/')
            gun_list.append(int(parts[0]))
            saat_list.append(int(parts[1]))
        
        df.index = pd.MultiIndex.from_arrays(
            [gun_list, saat_list],
            names=['Gün', 'Saat']
        )

    df_columns = df.columns.to_list()
    styled_df = df.style
    styled_df = auto_width_columns(styled_df)
    styled_df = styled_df.set_properties(**{'text-align': 'center'})
    styled_df = styled_df.format(thousands=',', precision=0)
    styled_df = styled_df.set_sticky(axis="index")

    numeric_cols = df.select_dtypes(include=[np.number]).columns

    def apply_color_by_change(col):
        styles = [''] * len(col)
        col_list = col.tolist()

        total_length = 24
        for i in range(len(col_list)):
            current_val = col_list[i]
            styles[i] = color_gradient(current_val, col_list)

        return styles


    for col in numeric_cols:
        styled_df = styled_df.apply(apply_color_by_change, subset=[col])

    def highlight_24h_border(row):
        saat = row.name[1] if isinstance(row.name, tuple) else None
        if saat == 23:
            return ['border-bottom: 2px solid black !important;'] * len(row)
        return [''] * len(row)
    
    styled_df = styled_df.apply(highlight_24h_border, axis=1)

    # 🔥 Table Styling - CSS ve JavaScript yerine Pandas styling kullan
    styled_df = styled_df.set_table_styles([
        {'selector': 'table', 'props': [
            ('width', '100%'), ('border-collapse', 'collapse'), ('table-layout', 'fixed')
        ]},
        {'selector': 'th', 'props': [
            ('width', '40px !important'), ('min-width', '20px !important'), 
            ('max-width', '100px !important'), ('font-size', '12px'), ('padding', '4px'),
            ('background-color', '#34495e'), ('color', 'white'), ('font-weight', '600'),
            ('position', 'relative'), ('white-space', 'nowrap'), ('text-align', 'left !important'),
            ('border', '1px solid #2c3e50'), ('line-height', '1.2')
        ]},
        {'selector':  'th.blank th.row_heading, th.level0', 'props': [
            ('width', '40px !important'), ('min-width', '20px !important'),
            ('max-width', '800px !important'), ('font-size', '10px')
        ]},
        {'selector': 'thead th.blank', 'props': [
            ('width', '40px !important'), 
            ('min-width', '20px !important'),
            ('max-width', '100px !important'), 
            ('font-size', '10px !important')
        ]},
        {'selector': 'th.col_heading.level0.col0', 'props': [
            ('width', '40px !important'), ('min-width', '20px !important'),
            ('max-width', '800px !important'), ('font-size', '10px')
        ]},
        {'selector': 'td', 'props': [
            ('width', '40px !important'), ('min-width', '20px !important'),
            ('max-width', '600px !important'), ('padding', '0px 2px'), 
            ('line-height', '0.5'), ('height', '12px'), ('white-space', 'nowrap'),
            ('font-size', '14px'), ('border-left', '2px solid #444444'),
            ('border-right', '2px solid #444444'), ('border-top', '1px solid #fff'),
            ('border-bottom', '1px solid #fff'), ('font-weight', 'bold')
        ]},
    ])

    # 🔥 Basitleştirilmiş JavaScript - CORS sorunu olmadan
    js_code = """
        <script type="text/javascript">
        (function() {
            function initResize() {
                const table = document.querySelector('.scrollable-table table');
                if (!table) {
                    console.log('Table not found, retrying...');
                    setTimeout(initResize, 200);
                    return;
                }
                
                const colHeaders = table.querySelectorAll('th.col_heading.level0');
                
                // Header'ları döndür ve 80px genişlik ver
                colHeaders.forEach((th) => {
                    const text = th.textContent.trim();
                    const span = document.createElement('span');
                    span.textContent = text;
                    span.style.cssText = 'display: inline-block; transform: rotate(-70deg); transform-origin: center center; font-size: 12px; white-space: nowrap; text-align:left;';
                    
                    th.style.cssText = 'writing-mode: horizontal-tb; height: 80px; vertical-align: center; text-align: left; width: 40px; min-width: 20px; position: relative; padding-left: 2px;';
                    
                    th.innerHTML = '';
                    th.appendChild(span);
                });
                
                // Tüm td'lere 80px genişlik ver
                const cells = table.querySelectorAll('tbody td');
                cells.forEach((td) => {
                    td.style.width = '40px';
                    td.style.minWidth = '40px';
                });
                
                // Resize mekanizması
                let isResizing = false;
                let currentTh = null;
                let startX = 0;
                let startWidth = 0;
                let colIndex = -1;
                
                const allHeaders = table.querySelectorAll('thead th');
                
                
                
                document.onmousemove = function(e) {
                    if (!isResizing) return;
                    
                    const diff = e.pageX - startX;
                    const newWidth = Math.max(80, startWidth + diff);
                    
                    currentTh.style.width = newWidth + 'px';
                    currentTh.style.minWidth = newWidth + 'px';
                    
                    // Aynı sütundaki tüm hücreleri resize et
                    const rows = table.querySelectorAll('tbody tr');
                    rows.forEach(function(row) {
                        const cell = row.querySelectorAll('td')[colIndex];
                        if (cell) {
                            cell.style.width = newWidth + 'px';
                            cell.style.minWidth = newWidth + 'px';
                        }
                    });
                };
                
                
                
                console.log('Resize initialized for ' + allHeaders.length + ' columns');
            }
            
            if (document.readyState === 'loading') {
                document.addEventListener('DOMContentLoaded', initResize);
            } else {
                initResize();
            }
        })();
        </script>
    """

    row_count = len(df)
    row_height = 16
    header_height = 160
    visible_rows = 48
    calculated_height = (visible_rows * row_height) + header_height

    html_content = f'''
    <style>
        * {{margin: 0; padding: 0; box-sizing: border-box;}}
        html, body {{width: 100%; overflow-x: auto;}}
        
        .table-wrapper {{
            display: flex; justify-content: center; width: 100vw;
            padding: 6px; box-sizing: border-box;
            margin-left: calc(-50vw + 50%); overflow: auto; max-height: 1500px;
        }}
        
        .scrollable-table {{
            max-height: {calculated_height}px; overflow: auto;
            border: 1px solid #ddd; border-radius: 5px;
            width: fit-content; max-width: calc(100vw);
        }}

        .scrollable-table table {{border-collapse: collapse; width: auto; table-layout: fixed;}}
        .scrollable-table thead tr:nth-child(2) {{display: none !important;}}
        .scrollable-table thead tr:first-child {{position: sticky; top: 0; z-index: 12;}}
        .scrollable-table thead th.row_heading,
        .scrollable-table thead th.blank,
        .scrollable-table thead th.index_name,
        .scrollable-table tbody th {{position: sticky !important; left: 0 !important;}}
        .scrollable-table tbody tr:hover {{background-color: #f5f5f5;}}
        .scrollable-table th,
        .scrollable-table td {{width: 100px;}}
    </style>
   
    {js_code}
    <div class="table-wrapper">
        <div class="scrollable-table">
            {styled_df.to_html()}
        </div>
    </div>
    '''

    if not lock_excel_button:
        html_content = create_excel_download_button_with_style(html_content, styled_df, f"{excel_file_name}.xlsx")
    display(HTML(html_content))
    return styled_df

## Gosterımler

### KGUP İLK

In [507]:
styled_df = styled_table(kgup_first_pivot, 'kgup_ilk', lock_excel_button=False)

### KGUP Son

In [510]:
styled_df = styled_table(kgup_toplam_pivot, 'kgup_son', lock_excel_button=False)

### KGUP SON \-  KGUP İLK

In [495]:
kgup_delta_kgup_first = create_delta_table(kgup_toplam_pivot, kgup_first_pivot)

# Bugünün tarihi ve saati
current_datetime = datetime.now(istanbul_tz)

# Bir saat öncesi (2 Şubat 2026 saat 11:33)
one_hour_before = current_datetime - timedelta(hours=1)

# Filtreleme için gün ve saat
filter_day = one_hour_before.day
filter_hour = one_hour_before.hour

kgup_delta_kgup_first

# MultiIndex'ten gün ve saat bilgilerini al
for idx in kgup_delta_kgup_first.index:
    gun = idx[0]  # Gün
    saat = idx[1]  # Saat
    
    if gun > filter_day or (gun == filter_day and saat >= filter_hour):
        kgup_delta_kgup_first.loc[idx, :] = 0.0

styled_df = styled_table(kgup_delta_kgup_first, 'kgupSon_fark_kgupİlk')

### ONCEKI GUN KGUP \-  KGUP SON

In [453]:
# styled_df = styled_table(kgup_day_earlier_pivot, 'test')

### ONCEKI GUN KGUP İLK \-  KGUP İLK

In [454]:
# Test et
# shifted_kgup = shift_pivot(kgup_first_pivot, 1)

# kgup_day_earlier_pivot = create_delta_table(shifted_kgup, kgup_first_pivot)

# styled_df = styled_table(kgup_day_earlier_pivot, 'test')

### EAK

In [455]:
styled_df = styled_table(aic_toplam_pivot, 'eak')

### Eak \- İLK KGUP

In [456]:
aic_delta_kgup_first = create_delta_table(aic_toplam_pivot, kgup_first_pivot)
styled_df = styled_table(aic_delta_kgup_first, 'eak_fark_ilkKgup')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3162b760-2d3a-4bdb-88e8-2b0bee1ebdd1' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>